In [1]:
import requests
from bs4 import BeautifulSoup as bs
import yfinance as yf
#ttcf = yf.Ticker("TTCF")

In [12]:
class CalculateAltZ:
    

    FIELDS = {
        'total_current_assets': 'Total Current Assets was ',
        'total_current_liabilities':'Total Current Liabilities was ',
        'total_assets': 'Total Assets was ',
        'retained_earnings': 'Retained Earnings was ',
        'pre_tax_income': 'Pre-Tax Income was ',
        'interest_expense': 'Interest Expense was ',
        'revenue': 'Revenue was ',
        'market_cap': 'Market Cap (Today) was ',
        'total_liabilities': 'Total Liabilities was '
    }

    MARKETS = ['NYSE', 'NAS']

    def __init__(self, ticker):
      
        self.ticker = ticker.upper()
        self.soup = bs(
            self.return_response(
                self.ticker
            ).text,
            'html.parser'
        )
        

        self.fs = self.financial_data
        self.start = self.fs.copy()

    @property
    def score(self):
        result =  f"""
        NYSE: {self.ticker}
        Altman Z-Score of ({round(self.calculate_score(), 4)})
        -----------------------------------------

        Working Capital   
        ------------------      : ({round(self.X1, 4)})
        Total Assets

        Retained Earnings 
        ------------------      : ({round(self.X2, 4)})
        Total Assets

        EBITA             
        ------------------      : ({round(self.X3, 4)})
        Total Assets

        Market Cap    
        ------------------      : ({round(self.X4, 4)})    
        Liabilities Book Value

        Revenue 
        ------------------      : ({round(self.X5, 4)})

        """

        print(result)
        for key, value in self.FIELDS.items():
            print(f"{value} {'${:,.2f}'.format(round(self.fs[key], 3))}")
     
    @classmethod
    def return_response(self, ticker):
   
        for m in self.MARKETS:
            url = f'https://www.gurufocus.com/term/zscore/{m}:{ticker}/Altman-Z-Score'
            print(url)
            response = requests.get(
                url,
            )

            if "does not have enough data to calculate Altman Z-Score" not in response.text:
                return response

    @property
    def financial_data(self):
        
        def clear_characters(row, name):
            row = row.split(
                    name
            )[-1].split("=")[-1]
            remove = ["$", ",", "Mil.", " "]
            for r in remove:
                row = row.replace(r, "")
            return float(row)*1000*1000
     
        z_data_dict = {}
        for row in self.soup.find_all("p")[19].text.split("\n"):
            for key, value in self.FIELDS.items():
                if value in row:
                    z_data_dict[key] = clear_characters(row, value)

        return z_data_dict

    @property
    def X1(self):
        working_capital = (
            self.fs['total_current_assets'] -
            self.fs['total_current_liabilities']
        )
        
        total_assets = self.fs['total_assets']

        return working_capital / total_assets

    @property
    def X2(self):
        return self.fs['retained_earnings'] / self.fs['total_assets']

    @property
    def X3(self):
        EBITA = (
            self.fs['pre_tax_income'] -
            self.fs['interest_expense']
        )

        total_assets = self.fs['total_assets']

        return EBITA / total_assets

    @property
    def X4(self):
        return self.fs['market_cap'] / self.fs['total_liabilities']

    @property
    def X5(self):
        return self.fs['revenue'] / self.fs['total_assets']

    def calculate_score(self):
        return 1.2 * self.X1 + 1.4 * self.X2 + 3.3 * self.X3 + 0.6 * self.X4 \
            + 1.0 * self.X5

    def reset(self):

        self.fs = self.financial_data

    def mod_i(self, amt, f, by="$"):
        amt = float(amt.replace(",", ""))
        value = self.fs.get(f)
        print(f"initial {f} value: {'${:,.2f}'.format(round(value, 3))}")
        if not value:
            return "Incorrect Field"
        else:
            if by == "$":
                i = amt
            elif by == "%":
                i = value * amt
            self.fs[f] += i
            print(f"modified {f} value: {'${:,.2f}'.format(round(self.fs[f], 3))}")

    def mod_d(self, amt, f, by="$"):
        amt = float(amt.replace(",", ""))
        value = self.fs.get(f)
        print(f"initial {f} value: {'${:,.2f}'.format(round(value, 3))}")
        if not value:
            return "Incorrect Field"
        else:
            if by == "$":
                i = amt
            elif by == "%":
                i = value * amt
            self.fs[f] -= i
            print(f"modified {f} value: {'${:,.2f}'.format(round(self.fs[f], 3))}")
        

    


In [15]:
# INSTANTIATE ALT Z CALCULATOR
x = CalculateAltZ('TTCF')

https://www.gurufocus.com/term/zscore/NYSE:TTCF/Altman-Z-Score
https://www.gurufocus.com/term/zscore/NAS:TTCF/Altman-Z-Score


In [16]:
#fields = {
#        'total_current_assets', 
#        'total_current_liabilities', 
#        'total_assets', 
#        'retained_earnings', 
#        'pre_tax_income', 
#        'interest_expense', 
#        'revenue', 
#        'market_cap', 
#        'total_liabilities', 
#}

x.reset()

retained_earnings = x.fs['retained_earnings']
interest_expense = x.fs['interest_expense']

print(interest_expense)
x.mod_d(".50", 'market_cap', "%")

increase_interest = interest_expense * .10

lost_earnings = 100 - retained_earnings - 100
print(retained_earnings)
# BORROW AMOUNT LOST
x.mod_i(str(lost_earnings), "total_liabilities", "$")
x.mod_i(str(lost_earnings+increase_interest), "total_liabilities", "$")

def decrease_revenue(amt):
        revenue = x.fs['revenue']
        i = revenue * amt
        x.mod_d(str(i), 'retained_earnings', "$")
        x.mod_d(str(i), 'pre_tax_income', "$")
        x.mod_d(str(i), 'revenue', "$")

decrease_revenue(.20)



x.score

-300000.0
initial market_cap value: $546,700,000.00
modified market_cap value: $273,350,000.00
-41700000.0
initial total_liabilities value: $60,900,000.00
modified total_liabilities value: $102,600,000.00
initial total_liabilities value: $102,600,000.00
modified total_liabilities value: $144,270,000.00
initial retained_earnings value: $-41,700,000.00
modified retained_earnings value: $-88,300,000.00
initial pre_tax_income value: $-47,100,000.00
modified pre_tax_income value: $-93,700,000.00
initial revenue value: $233,000,000.00
modified revenue value: $186,400,000.00

        NYSE: TTCF
        Altman Z-Score of (0.7136)
        -----------------------------------------

        Working Capital   
        ------------------      : (0.4294)
        Total Assets

        Retained Earnings 
        ------------------      : (-0.3377)
        Total Assets

        EBITA             
        ------------------      : (-0.3572)
        Total Assets

        Market Cap    
        ----------